In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('DeadFuelLag')\
.config("spark.jars.packages", "com.microsoft.ml.spark:mmlspark_2.11:1.0.0-rc1")\
.config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven").getOrCreate()

In [3]:
import mmlspark

In [2]:
# import pandas as pd
# df = pd.read_parquet('gridMet.parquet.gz')
# #df["surface_downwelling_shortwave_flux_in_air_W_m-2"] = df["surface_downwelling_shortwave_flux_in_air_W m-2"]
# #del df["surface_downwelling_shortwave_flux_in_air_W m-2"]
# df["wind_from_direction_Degrees_Clockwise_from_north"] = df["wind_from_direction_Degrees Clockwise_from_north"]
# del df["wind_from_direction_Degrees Clockwise_from_north"]
# df.to_parquet('gridMet.parquet.gz')

In [4]:
df = spark.read.parquet("gridMet.parquet.gz").where(F.col("precipitation_amount_mm").isNotNull())

In [5]:
df.printSchema()

root
 |-- precipitation_amount_mm: double (nullable = true)
 |-- relative_humidity_%: double (nullable = true)
 |-- specific_humidity_kg/kg: double (nullable = true)
 |-- wind_speed_m/s: double (nullable = true)
 |-- max_air_temperature_K: double (nullable = true)
 |-- min_air_temperature_K: double (nullable = true)
 |-- burning_index_g_Unitless: double (nullable = true)
 |-- dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- energy_release_component-g_Unitless: double (nullable = true)
 |-- potential_evapotranspiration_mm: double (nullable = true)
 |-- mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- surface_downwelling_shortwave_flux_in_air_W_m-2: double (nullable = true)
 |-- wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [6]:
windowSpec = Window.partitionBy(F.col("latitude"), F.col("longitude")).orderBy(F.col("date").desc())
df = df.withColumnRenamed("surface_downwelling_shortwave_flux_in_air_W_m-2","surface_downwelling_shortwave_flux_in_air_W m-2")
df = df.withColumn("cumLag", F.lit(0))
for i in range(1,8):
    df = df.withColumn("lag-"+str(i),F.col("dead_fuel_moisture_1000hr_Percent")- F.lag(F.col("dead_fuel_moisture_1000hr_Percent"), i).over(windowSpec))
    df = df.withColumn("cumLag", F.when((F.col("lag-"+str(i))<= 0) & (F.col("cumLag") == i-1), i ).otherwise(F.col("cumLag")))
df = df.withColumn("lag-Test",F.lag(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec))

In [13]:
def AggFunctions(df, cols):
    for idx, col in enumerate(cols):
        df = df.withColumn(col+"_mean_7_days", F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-7,0)))
        df = df.withColumn(col+"_mean_14_days", F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-14,0)))
        df = df.withColumn(col+"_mean_30_days", F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-30,0)))
        df = df.withColumn(col+"_mean_6_months", F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-182,0)))
        df = df.withColumn(col+"_mean_1_year", F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-365,0)))
        
        df = df.withColumn(col+"_max_7_days", F.max(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-7,0)))
        df = df.withColumn(col+"_max_14_days", F.max(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-14,0)))
        df = df.withColumn(col+"_max_30_days", F.max(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-30,0)))
        df = df.withColumn(col+"_max_6_months", F.max(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-182,0)))
        df = df.withColumn(col+"_max_1_year", F.max(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-365,0)))
    return df
        
agg_cols = ['mean_vapor_pressure_deficit_kPa','relative_humidity_%',
                                             'specific_humidity_kg/kg','wind_speed_m/s','max_air_temperature_K',
                                             'dead_fuel_moisture_100hr_Percent','dead_fuel_moisture_1000hr_Percent']
df = AggFunctions(df,agg_cols)
        

In [7]:
df = df.withColumn("movingAvg",F.avg(F.col("dead_fuel_moisture_1000hr_Percent")).over(windowSpec.rowsBetween(-7,0)))

[Row(precipitation_amount_mm=0.0, relative_humidity_%=27.200000000000003, specific_humidity_kg/kg=0.005090000000000001, wind_speed_m/s=1.8, max_air_temperature_K=292.6, min_air_temperature_K=274.2, burning_index_g_Unitless=17.0, dead_fuel_moisture_100hr_Percent=19.1, dead_fuel_moisture_1000hr_Percent=22.1, energy_release_component-g_Unitless=19.0, potential_evapotranspiration_mm=1.7000000000000002, mean_vapor_pressure_deficit_kPa=0.67, surface_downwelling_shortwave_flux_in_air_W m-2=121.4, wind_from_direction_Degrees_Clockwise_from_north=92.0, date=datetime.datetime(2019, 12, 30, 16, 0), latitude=33.025000000000006, longitude=-116.97499996666667, cumLag=0, lag-1=None, lag-2=None, lag-3=None, lag-4=None, lag-5=None, lag-6=None, lag-7=None, lag-Test=None, movingAvg=22.1, mean_vapor_pressure_deficit_kPa_mean_7_days=22.1, mean_vapor_pressure_deficit_kPa_mean_14_days=22.1, mean_vapor_pressure_deficit_kPa_mean_30_days=22.1, mean_vapor_pressure_deficit_kPa_mean_6_months=22.1, mean_vapor_press

In [8]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [14]:
df.printSchema()

root
 |-- precipitation_amount_mm: double (nullable = true)
 |-- relative_humidity_%: double (nullable = true)
 |-- specific_humidity_kg/kg: double (nullable = true)
 |-- wind_speed_m/s: double (nullable = true)
 |-- max_air_temperature_K: double (nullable = true)
 |-- min_air_temperature_K: double (nullable = true)
 |-- burning_index_g_Unitless: double (nullable = true)
 |-- dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- energy_release_component-g_Unitless: double (nullable = true)
 |-- potential_evapotranspiration_mm: double (nullable = true)
 |-- mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- surface_downwelling_shortwave_flux_in_air_W m-2: double (nullable = true)
 |-- wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- cumLag: integer (nul

In [10]:
features = [
 'precipitation_amount_mm',
 'relative_humidity_%',
 'specific_humidity_kg/kg',
 'surface_downwelling_shortwave_flux_in_air_W m-2',
 'wind_from_direction_Degrees_Clockwise_from_north',
 'wind_speed_m/s',
 'max_air_temperature_K',
 'min_air_temperature_K',
 'burning_index_g_Unitless',
 'dead_fuel_moisture_100hr_Percent',
 'dead_fuel_moisture_1000hr_Percent',
 'energy_release_component-g_Unitless',
 'potential_evapotranspiration_mm',
 'mean_vapor_pressure_deficit_kPa',
 'cumLag'
]
assembler = VectorAssembler(
    inputCols= features,
    outputCol='features'
)
train_df = df.withColumn("label", F.rand().cast("int"))
train_df = assembler.transform(train_df)

lr = LogisticRegression(featuresCol='features', labelCol='label')
model = lr.fit(train_df)

In [12]:
## TODO see if mmlspark works on EMR
# from mmlspark.lightgbm import LightGBMRegressor
# model = LightGBMRegressor(objective='quantile',
#                           alpha=0.2,
#                           learningRate=0.3,
#                           numLeaves=31).fit(train_df)

Py4JJavaError: An error occurred while calling o383.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 238, localhost, executor driver): org.apache.spark.memory.SparkOutOfMemoryError: Unable to acquire 16384 bytes of memory, got 0
	at org.apache.spark.memory.MemoryConsumer.throwOom(MemoryConsumer.java:157)
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:98)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeInMemorySorter.<init>(UnsafeInMemorySorter.java:128)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:161)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:128)
	at org.apache.spark.sql.execution.ExternalAppendOnlyUnsafeRowArray.add(ExternalAppendOnlyUnsafeRowArray.scala:115)
	at org.apache.spark.sql.execution.window.WindowExec